In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(self):
        config_filepath = CONFIG_FILE_PATH

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source=config.source,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [11]:
import os
import urllib.request as request
import zipfile
from src.hybrid_recommender import logger
from src.hybrid_recommender.utils.common import get_size
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.api = KaggleApi()
        self.api.authenticate()

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading dataset from Kaggle: {self.config.source}")
            self.api.dataset_download_files(
                self.config.source,
                path=self.config.root_dir,
                unzip=False
            )
            temp_zip = os.path.join(self.config.root_dir, f"{self.config.source.split('/')[-1]}.zip")
            if os.path.exists(temp_zip):
                os.rename(temp_zip, self.config.local_data_file)
            
            logger.info(f"Downloaded dataset to: {self.config.local_data_file}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted data to: {unzip_path}")
  
    def merge_data(self) -> pd.DataFrame:
        """
        Merges data from Books, Ratings, and Users CSV files into a single DataFrame
        
        Returns:
            pd.DataFrame: Merged dataset with book details, user details, and ratings
        """
        try:
            # Read individual files
            books_path = os.path.join(self.config.unzip_dir, "Books.csv")
            ratings_path = os.path.join(self.config.unzip_dir, "Ratings.csv")
            users_path = os.path.join(self.config.unzip_dir, "Users.csv")
            
            logger.info(f"Reading data files from {self.config.unzip_dir}")
            books_df = pd.read_csv(books_path, low_memory=False, encoding='latin1')
            ratings_df = pd.read_csv(ratings_path, low_memory=False, encoding='latin1')
            users_df = pd.read_csv(users_path, low_memory=False, encoding='latin1')
            
            logger.info(f"Books shape: {books_df.shape}, Ratings shape: {ratings_df.shape}, Users shape: {users_df.shape}")
            
            # First merge ratings with books on ISBN
            logger.info("Merging ratings with books...")
            book_ratings = pd.merge(
                ratings_df,
                books_df,
                on='ISBN',
                how='left'
            )
            
            # Then merge with users on User-ID
            logger.info("Merging with users...")
            final_df = pd.merge(
                book_ratings,
                users_df,
                on='User-ID',
                how='left'
            )
            
            logger.info(f"Merged dataset shape: {final_df.shape}")
            
            # Save merged data
            merged_path = os.path.join(self.config.root_dir, "merged_data.csv")
            final_df.to_csv(merged_path, index=False)
            logger.info(f"Saved merged data to {merged_path}")
            
            return final_df
            
        except Exception as e:
            logger.error(f"Error merging datasets: {e}")
            raise e

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-16 10:35:20,810: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-16 10:35:20,811: INFO: common: created directory at: artifacts]
[2025-06-16 10:35:20,812: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-16 10:35:20,813: INFO: 1284476475: Downloading dataset from Kaggle: arashnic/book-recommendation-dataset]
Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
[2025-06-16 10:35:49,574: INFO: 1284476475: Downloaded dataset to: artifacts/data_ingestion/data.zip]
[2025-06-16 10:35:49,993: INFO: 1284476475: Extracted data to: artifacts/data_ingestion/data]
